In [1]:
import os
import numpy as np
import pandas as pd
from pandas import unique
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
from math import e
from scipy.optimize import minimize
from scipy.stats import poisson
from scipy.special import gamma
from scipy.special import gammaln
from scipy.special import factorial
from collections import Counter

In [2]:
os.chdir(r'C:\buan6383\data')
rawbooks = pd.read_csv('articles.csv')
rawbooks

,articles,female,married,kids,prestige,menpubs
0,0,0,1,0,2.52,7
1,0,1,0,0,2.05,6
2,0,1,0,0,3.75,6
3,0,0,1,1,1.18,3
4,0,1,0,0,3.75,26
...,...,...,...,...,...,...
910,11,0,1,2,2.86,7
911,12,0,1,1,4.29,35
912,12,0,1,1,1.86,5
913,16,0,1,0,1.74,21


# Poisson

In [3]:
arti=rawbooks
b8= np.array(arti['articles'])
a81= np.array(arti['female'])
a82= np.array(arti['married'])
a83= np.array(arti['kids'])
a84= np.array(arti['prestige'])
a85= np.array(arti['menpubs'])



In [4]:
def PoisReg8(params,a81,a82,a83,a84,a85,b8):
    lam0, beta1, beta2, beta3, beta4, beta5 = params
    pll   = 0
    ll1 = 0
    ll2 = 0
    ll3 = 0
    for i in range(len(a81)):
        expformula=np.exp(beta1*a81[i]+beta2*a82[i]+beta3*a83[i]+beta4*a84[i]+beta5*a85[i])
        lam=lam0*expformula
        kfact = float(math.factorial(int(b8[i])))
        ll1 = ll1 + b8[i]*np.log(lam)
        ll2 = ll2 + lam
        ll3 = ll3 + np.log(kfact)
    pll = ll1-ll2-ll3
    return (-1)*pll
  


Pois_result8 = minimize(PoisReg8, args=(a81,a82,a83,a84,a85,b8),x0=np.array((1,1,-1,1,-1,0.001)),bounds=[(0.00001,None),(None,None),(None,None),(None,None),(None,None),(None,None)],tol=1e-10,options={'ftol':1e-8})

print(Pois_result8)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1651.0563270786183
        x: [ 1.356e+00 -2.246e-01  1.550e-01 -1.847e-01  1.283e-02
             2.554e-02]
      nit: 45
      jac: [ 5.662e-02  9.095e-03  1.496e-02  1.006e-01  2.600e-01
             8.306e-01]
     nfev: 406
     njev: 58
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>


In [5]:
ll1=0
ll2=0
ll3=0
    
lam0 =  Pois_result8.x[0]
beta1 = Pois_result8.x[1]
beta2 = Pois_result8.x[2]
beta3 = Pois_result8.x[3]
beta4 = Pois_result8.x[4]
beta5 = Pois_result8.x[5]


for i in range(len(a81)):
    lam=lam0*math.exp(beta1*a81[i]+beta2*a82[i]+beta3*a83[i]+beta4*a84[i]+beta5*a85[i])
    kfact = float(math.factorial(float(b8[i])))
    ll1 = ll1 + b8[i]*np.log(lam)
    ll2 = ll2 + lam
    ll3 = ll3 + np.log(kfact)
        
        
pll = ll1-ll2-ll3
        
print("lam0: {}".format(lam0))
print("beta1: {}".format(beta1))
print("beta2: {}".format(beta2))
print("beta3: {}".format(beta3))
print("beta4: {}".format(beta4))
print("beta5: {}".format(beta5))

print("log likelihood: {}".format(pll))
q8k = len(Pois_result8.x)
print(q8k)
q8n = len(arti)
q8n

lam0: 1.3563079599317214
beta1: -0.22464867487897558
beta2: 0.15503240593346085
beta3: -0.18474839498028656
beta4: 0.012828689593332158
beta5: 0.025541491493694373
log likelihood: -1651.0563270786183
6


C:\Users\sruja\AppData\Local\Temp\ipykernel_1388\1400655183.py:15: DeprecationWarning: Using factorial() with floats is deprecated
  kfact = float(math.factorial(float(b8[i])))


915

# Question 9

In [6]:
def NBDReg9(params,x1,x2,x3,x4,x5,y):
    n,alpha,nbeta1, nbeta2, nbeta3, nbeta4, nbeta5= params
    nll=0
    nll1=0
    nll2=0
    nll3=0
    nll4=0
    for i in range(len(x1)):
        beta=math.exp(nbeta1*x1[i]+nbeta2*x2[i]+nbeta3*x3[i]+nbeta4*x4[i]+nbeta5*x5[i])
        nll1=nll1+np.log(math.gamma(n+int(y[i])))
        nll2=nll2+np.log(math.gamma(n))+np.log(factorial(int(y[i])))
        nll3=nll3+n*(np.log(alpha)-np.log(alpha+beta))
        nll4=nll4+int(y[i])*(np.log(beta)-np.log(alpha+beta))
    nll=nll1-nll2+nll3+nll4
    return nll*(-1)

In [16]:
NBD_result9 = minimize(NBDReg9, args=(a81,a82,a83,a84,a85,b8),x0=np.array((1,1,-1,1,-1,1,0.001)),bounds=[(0.00001,None),(0.00001,None),(None,None),(None,None),(None,None),(None,None),(None,None)],tol=1e-10,options={'ftol':1e-8})

nll=0
nll1=0
nll2=0
nll3=0
nll4=0   
n = NBD_result9.x[0]
alpha = NBD_result9.x[1]
beta1 = NBD_result9.x[2]
beta2 = NBD_result9.x[3]
beta3 = NBD_result9.x[4]
beta4 = NBD_result9.x[5]
beta5 = NBD_result9.x[6]
print(NBD_result9)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1560.9583467794657
        x: [ 2.265e+00  1.753e+00 -2.164e-01  1.507e-01 -1.765e-01
             1.517e-02  2.908e-02]
      nit: 48
      jac: [ 7.822e-03 -7.731e-04  9.845e-03  2.797e-02 -6.457e-03
            -7.906e-02 -6.521e-02]
     nfev: 440
     njev: 55
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>


In [8]:
for i in range(len(a81)):
    beta=math.exp(beta1*a81[i]+beta2*a82[i]+beta3*a83[i]+beta4*a84[i]+beta5*a85[i])
    nll1=nll1+np.log(math.gamma(n+int(b8[i])))
    nll2=nll2+np.log(math.gamma(n))+np.log(factorial(int(b8[i])))
    nll3=nll3+n*(np.log(alpha)-np.log(alpha+beta))
    nll4=nll4+int(b8[i])*(np.log(beta)-np.log(alpha+beta))
nll=nll1-nll2+nll3+nll4

In [9]:
print("n: {}".format(n))
print("alpha: {}".format(alpha))
print("beta1: {}".format(beta1))
print("beta2: {}".format(beta2))
print("beta3: {}".format(beta3))
print("beta4: {}".format(beta4))
print("beta5: {}".format(beta5))

print("log likelihood: {}".format(nll))
q9nll = nll
q9k = len(NBD_result9.x)
print(q9k)
q9n = len(arti)
q9n

n: 2.264839359215258
alpha: 1.7527458418322077
beta1: -0.21635817969672408
beta2: 0.15066810328789315
beta3: -0.17647809371037154
beta4: 0.015174051948067533
beta5: 0.029083742599197177
log likelihood: -1560.9583467794657
7


915

# Question 3


In [10]:
def NBDReg91(params,x1,x2,x3,x4,x5,y):
    n,alpha,nbeta1, nbeta2, nbeta3, nbeta4, nbeta5,pi= params
    nll=0
    nll1=0
    nll2=0
    nll3=0
    nll4=0
    zero_val=0
    pi_val=0
    for i in range(len(x1)):
        beta=math.exp(nbeta1*x1[i]+nbeta2*x2[i]+nbeta3*x3[i]+nbeta4*x4[i]+nbeta5*x5[i])
        if y[i]==0:
            zero_val=zero_val+np.log(pi+(1-pi)*((alpha/(alpha+beta))**n))
        else: 
            nll1=nll1+np.log(math.gamma(n+int(y[i])))
            nll2=nll2+np.log(math.gamma(n))+np.log(factorial(int(y[i])))
            nll3=nll3+n*(np.log(alpha)-np.log(alpha+beta))
            nll4=nll4+int(y[i])*(np.log(beta)-np.log(alpha+beta))
            pi_val=pi_val+np.log(1-pi)
    nll=nll1-nll2+nll3+nll4+pi_val+zero_val
    return nll*(-1)

In [11]:
NBD_result91 = minimize(NBDReg91, args=(a81,a82,a83,a84,a85,b8),x0=np.array((1,1,-1,1,-1,1,0.01,0.3)),bounds=[(0.00001,None),(0.00001,None),(None,None),(None,None),(None,None),(None,None),(None,None),(0.000001,0.9999999)],tol=1e-10,options={'ftol':1e-8})
print(NBD_result9)
nll=0
nll1=0
nll2=0
nll3=0
nll4=0   
n = NBD_result91.x[0]
alpha = NBD_result91.x[1]
beta1 = NBD_result91.x[2]
beta2 = NBD_result91.x[3]
beta3 = NBD_result91.x[4]
beta4 = NBD_result91.x[5]
beta5 = NBD_result91.x[6]
pi=NBD_result91.x[7]
q91k = len(NBD_result91.x)
q91nll=-NBD_result91.fun

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1560.9583467794657
        x: [ 2.265e+00  1.753e+00 -2.164e-01  1.507e-01 -1.765e-01
             1.517e-02  2.908e-02]
      nit: 48
      jac: [ 7.822e-03 -7.731e-04  9.845e-03  2.797e-02 -6.457e-03
            -7.906e-02 -6.521e-02]
     nfev: 440
     njev: 55
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>


In [12]:
print("n: {}".format(n))
print("alpha: {}".format(alpha))
print("beta1: {}".format(beta1))
print("beta2: {}".format(beta2))
print("beta3: {}".format(beta3))
print("beta4: {}".format(beta4))
print("beta5: {}".format(beta5))
print("pi:{:.8f}".format(pi))
print(q91k)
print(q91nll)

n: 2.2644401537288696
alpha: 1.7527444490669728
beta1: -0.21641285699968965
beta2: 0.150483474114976
beta3: -0.17640787652409876
beta4: 0.015270476393750766
beta5: 0.02908230716174971
pi:0.00000100
8
-1560.9583876407066


# 3


In [13]:
#Poisson Regression 
preg_aic = 2*q8k-2*pll
preg_bic = q8k*math.log(q8n)-2*pll
print(preg_aic)
print(preg_bic)
print(pll)
print(q8k)
print("AIC of Poisson Regression : {}".format(preg_aic))
print("BIC of Poisson Regression : {}".format(preg_bic))
print("LL of Poisson Regression : {}".format(pll))
print("Parameters of Poisson Regression : {}".format(q8k))

3314.1126541572366
3343.0261985488896
-1651.0563270786183
6
AIC of Poisson Regression : 3314.1126541572366
BIC of Poisson Regression : 3343.0261985488896
LL of Poisson Regression : -1651.0563270786183
Parameters of Poisson Regression : 6


In [14]:
#NBD Regression
NBDreg_aic = 2*q9k-2*q9nll
NBDreg_bic = q9k*math.log(q9n)-2*q9nll
print(NBDreg_aic)
print(NBDreg_bic)
print(q9nll)
print(q9k)
print("AIC of NBD Regression : {}".format(NBDreg_aic))
print("BIC of NBD Regression : {}".format(NBDreg_bic))
print("LL of NBD Regression : {}".format(q9nll))
print("Parameters of NBD Regression : {}".format(q9k))

3135.9166935589315
3169.64916201586
-1560.9583467794657
7
AIC of NBD Regression : 3135.9166935589315
BIC of NBD Regression : 3169.64916201586
LL of NBD Regression : -1560.9583467794657
Parameters of NBD Regression : 7


In [15]:
#Zero-inflated NBD Regression
Zi_NBDreg_aic = 2*q91k-2*q91nll
Zi_NBDreg_bic = q91k*math.log(q9n)-2*q91nll
print(Zi_NBDreg_aic)
print(Zi_NBDreg_bic)
print(q9nll)
print(q91k)
print("AIC of Zi_NBD Regression : {}".format(Zi_NBDreg_aic))
print("BIC of Zi_NBD Regression : {}".format(Zi_NBDreg_bic))
print("LL of Zi_NBD Regression : {}".format(q91nll))
print("Parameters of Zi_NBD Regression : {}".format(q91k))

3137.9167752814133
3176.4681678036172
-1560.9583467794657
8
AIC of Zi_NBD Regression : 3137.9167752814133
BIC of Zi_NBD Regression : 3176.4681678036172
LL of Zi_NBD Regression : -1560.9583876407066
Parameters of Zi_NBD Regression : 8
